In [36]:
from pyspark.sql import SparkSession, Window
import pyspark.sql.functions as f
from pyspark.sql.types import *
from functools import reduce


In [2]:
! pyspark --version

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 3.1.2
      /_/
                        
Using Scala version 2.12.10, OpenJDK 64-Bit Server VM, 1.8.0_312
Branch HEAD
Compiled by user centos on 2021-05-24T04:27:48Z
Revision de351e30a90dd988b133b3d00fa6218bfcaba8b8
Url https://github.com/apache/spark
Type --help for more information.


In [3]:
data_path = '/home/daniel/Desktop/topics/datasets/smart homes' # en esta carpeta deben encontrarse los 4 ficheros

In [4]:
spark = SparkSession.builder.master("local[*]")\
          .appName("practica 2 Ecosistema Spark parte 3")\
          .getOrCreate()

22/01/28 19:43:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [52]:
home_data = spark.read.csv(data_path, header=True, inferSchema=True, encoding='latin1')

In [53]:
newColumns = []
oldColumns = home_data.schema.names
for ind, name in enumerate(oldColumns):
    try:
        newColumns.append(name[:name.index('[')].strip())
    except ValueError:
        newColumns.append(name)
    
home_data = reduce(lambda data, idx: data.withColumnRenamed(oldColumns[idx], newColumns[idx]), range(len(oldColumns)), home_data)

In [55]:
home_data.printSchema()

root
 |-- time: string (nullable = true)
 |-- use: double (nullable = true)
 |-- gen: double (nullable = true)
 |-- House overall: double (nullable = true)
 |-- Dishwasher: double (nullable = true)
 |-- Furnace 1: double (nullable = true)
 |-- Furnace 2: double (nullable = true)
 |-- Home office: double (nullable = true)
 |-- Fridge: double (nullable = true)
 |-- Wine cellar: double (nullable = true)
 |-- Garage door: double (nullable = true)
 |-- Kitchen 12: double (nullable = true)
 |-- Kitchen 14: double (nullable = true)
 |-- Kitchen 38: double (nullable = true)
 |-- Barn: double (nullable = true)
 |-- Well: double (nullable = true)
 |-- Microwave: double (nullable = true)
 |-- Living room: double (nullable = true)
 |-- Solar: double (nullable = true)
 |-- temperature: double (nullable = true)
 |-- icon: string (nullable = true)
 |-- humidity: double (nullable = true)
 |-- visibility: double (nullable = true)
 |-- summary: string (nullable = true)
 |-- apparentTemperature: double (

# Weather Features Information

- temperature: physical quantity expressing hot and cold

- humidity: concentration of water vapour present in air
- visibility: meteorological optical range which is defined as the length of atmosphere over which a beam of light travels before its luminous flux is reduced to 5% of its original value
- apparentTemperature: temperature equivalent perceived by humans, caused by the combined effects of air temperature, relative humidity and wind speed
- pressure: Falling air pressure indicates that bad weather is coming, while rising air pressure indicates good weather
- windSpeed: fundamental atmospheric quantity caused by air moving from high to low pressure, usually due to changes in temperature
- cloudCover: fraction of the sky obscured by clouds when observed from a particular location
- windBearing: In meteorology, an azimuth of 000° is used only when no wind is blowing, while 360° means the wind is from the North. True Wind Direction True North is represented on a globe as the North Pole. All directions relative to True North may be called "true bearings."
- dewPoint: the atmospheric temperature (varying according to pressure and humidity) below which water droplets begin to condense and dew can form
- precipProbability: measure of the probability that at least some minimum quantity of precipitation will occur within a specified forecast period and location
- precipIntensity: measure of the amount of rain that falls over time

In [63]:
home_data.select([f.mean(col) for col in newColumns]).show()

+--------------+------------------+-------------------+------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------------+--------------------+--------------------+-------------------+-------------------+--------------------+--------------------+--------------------+-------------------+-------------------+-----------------+---------+------------------+-----------------+------------+------------------------+-----------------+------------------+-------------------+------------------+--------------------+-----------------+----------------------+
|     avg(time)|          avg(use)|           avg(gen)|avg(House overall)|    avg(Dishwasher)|     avg(Furnace 1)|     avg(Furnace 2)|   avg(Home office)|        avg(Fridge)|    avg(Wine cellar)|    avg(Garage door)|     avg(Kitchen 12)|    avg(Kitchen 14)|    avg(Kitchen 38)|           avg(Barn)|           avg(Well)|      avg(Microwave)|   avg(Living room)|         avg(Solar